In [50]:
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import uproot
import os
import glob
from itertools import chain
from copy import deepcopy
import plotly.graph_objs as go
import plotly.express as px
from tqdm import tqdm
from particle import Particle, InvalidParticle, ParticleNotFound
from dataclasses import dataclass, field
from typing import Tuple
import scipy.optimize as opt

In [21]:
lumi_sf = 1/10
tungsten_density_gcm3 = 19.3 # g/cm^3
nominal_sheet_area_cm2 = 25 * 30 # cm^2
nominal_tungsten_thickness = 0.09 # cm 
nominal_length_cm = 105 # cm
nominal_number_of_layers = 132
data_fpath = "/home/bewilson/FASERvSi_G4/GeantOutput/CombinedLightCharm6800fb.root"

In [3]:
data = uproot.open(f"{data_fpath}:Hits")

In [4]:
events = data.arrays(data.keys())

In [11]:
cc_data = events[events['isCC'] == True]
nc_data = events[events['isCC'] == False]
cc_el_data = events[(events['isCC'] == True) & (abs(events['nu_pdgc']) == 12)]
cc_mu_data = events[(events['isCC'] == True) & (abs(events['nu_pdgc']) == 14)]
cc_tau_data = events[(events['isCC'] == True) & (abs(events['nu_pdgc']) == 16)]

In [12]:
n_nc_events = len(nc_data)
n_cc_el_events = len(cc_el_data)
n_cc_mu_events = len(cc_mu_data)
n_cc_tau_events = len(cc_tau_data)

In [13]:
print(f"Number of raw events: {len(events)}")

Number of raw events: 46042


In [38]:
print("Baseline Setup - 132 SCT | 132 Tungsten | 0.9 mm")
print(f"{'-' * 60}")
print(f"Number of Neutral current events: {n_nc_events*lumi_sf:.2f}")
print(f"Number of CC electron neutrino events: {n_cc_el_events*lumi_sf:.2f}")
print(f"Number of CC muon neutrino events: {n_cc_mu_events*lumi_sf:.2f}")
print(f"Number of CC tau neutrino events: {n_cc_tau_events*lumi_sf:.2f}")
print("\n")
nominal_tungsten_mass_g = tungsten_density_gcm3 * nominal_sheet_area_cm2 * nominal_tungsten_thickness * nominal_number_of_layers
print(f"Nominal target mass = {nominal_tungsten_mass_g/1000:.2f} kg")

Baseline Setup - 132 SCT | 132 Tungsten | 0.9 mm
------------------------------------------------------------
Number of Neutral current events: 1081.10
Number of CC electron neutrino events: 539.00
Number of CC muon neutrino events: 2971.90
Number of CC tau neutrino events: 12.20


Nominal target mass = 171.96 kg


In [37]:
print("Alternative 1 - 100 SCT | 100 Tungsten | 3.4 mm")
print(f"{'-' * 60}")
tungsten_mass_g = tungsten_density_gcm3 * nominal_sheet_area_cm2 * 0.34 * 100

alt_1_sf = (tungsten_mass_g / nominal_tungsten_mass_g) * lumi_sf

print(f"New mass / nominal = {tungsten_mass_g / nominal_tungsten_mass_g:.2f}")

print(f"Number of Neutral current events: {n_nc_events*alt_1_sf:.2f}")
print(f"Number of CC electron neutrino events: {n_cc_el_events*alt_1_sf:.2f}")
print(f"Number of CC muon neutrino events: {n_cc_mu_events*alt_1_sf:.2f}")
print(f"Number of CC tau neutrino events: {n_cc_tau_events*alt_1_sf:.2f}")
print("\n")

print(f"Nominal target mass = {tungsten_mass_g/1000:.2f} kg")

Alternative 1 - 100 SCT | 100 Tungsten | 3.4 mm
------------------------------------------------------------
New mass / nominal = 2.86
Number of Neutral current events: 3094.06
Number of CC electron neutrino events: 1542.59
Number of CC muon neutrino events: 8505.44
Number of CC tau neutrino events: 34.92


Nominal target mass = 492.15 kg


In [48]:
thickness_1tonne = 1000000 / (tungsten_density_gcm3 * nominal_sheet_area_cm2 * 100)
print(f"Sheet thickness for 100 layer and 1 Tonne: {thickness_1tonne:.2f} cm")
det_length = 100 * (0.708 + 0.69)
print(f"Detector length = {det_length:.2f} cm")

Sheet thickness for 100 layer and 1 Tonne: 0.69 cm
Detector length = 139.80 cm


In [45]:
print("Alternative 2 - 100 SCT | 1000 kg Tungsten | 6.9 mm")
print(f"{'-' * 60}")
tungsten_mass_g = tungsten_density_gcm3 * nominal_sheet_area_cm2 * 0.69 * 100

alt_2_sf = (tungsten_mass_g / nominal_tungsten_mass_g) * lumi_sf

print(f"New mass / nominal = {tungsten_mass_g / nominal_tungsten_mass_g:.2f}")

print(f"Number of Neutral current events: {n_nc_events*alt_2_sf:.2f}")
print(f"Number of CC electron neutrino events: {n_cc_el_events*alt_2_sf:.2f}")
print(f"Number of CC muon neutrino events: {n_cc_mu_events*alt_2_sf:.2f}")
print(f"Number of CC tau neutrino events: {n_cc_tau_events*alt_2_sf:.2f}")
print("\n")

print(f"Nominal target mass = {tungsten_mass_g/1000:.2f}")

Alternative 2 - 100 SCT | 1000 kg Tungsten | 6.9 mm
------------------------------------------------------------
New mass / nominal = 5.81
Number of Neutral current events: 6279.12
Number of CC electron neutrino events: 3130.56
Number of CC muon neutrino events: 17261.04
Number of CC tau neutrino events: 70.86


Nominal target mass = 998.77


In [85]:
def func_to_minimise(args, target_mass=1000000, target_length=105, mass_tol=5000):
    thickness, nlayers = args
    det_length = nlayers * (thickness + 0.709)
    det_masss = nlayers * thickness * tungsten_density_gcm3 * nominal_sheet_area_cm2
    
    if det_length > target_length: 
        # print(f"{thickness:.3f} {nlayers} --- Too long! -- {det_length} cm")
        return 1e10
    # if abs(target_mass - det_masss) > mass_tol: 
    #     print(f"{thickness:.3f} {nlayers} --- Not the right mass! -- {det_masss/1000:.2f} kg")
    #     return 1e10
    
    # print(abs(target_mass - det_masss))
    return abs(target_mass - det_masss)

In [97]:
initial_guess = [0.69, 75]

# Minimize the function
result = opt.fmin(func_to_minimise, initial_guess, args=(1000000, 105, 5000))
                  
print(f"Optimal thickness: {result[0]:.2f} cm")
print(f"Optimal number of layers: {result[1]:.0f}")
print(f"Detector length = {round(result[1]) * (result[0] + 0.709):.2f} cm")
print(f"Detector mass = {(result[1] * result[0] * tungsten_density_gcm3 * nominal_sheet_area_cm2)/1000:.2f} kg")

Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 83
         Function evaluations: 155
Optimal thickness: 1.53 cm
Optimal number of layers: 45
Detector length = 100.81 cm
Detector mass = 1000.00 kg


In [101]:
print("Alternative 2 - 45 SCT | 1000 kg Tungsten | 150 mm")
print(f"{'-' * 60}")
tungsten_mass_g = tungsten_density_gcm3 * nominal_sheet_area_cm2 * 1.53 * 45

alt_3_sf = (tungsten_mass_g / nominal_tungsten_mass_g) * lumi_sf

print(f"New mass / nominal = {tungsten_mass_g / nominal_tungsten_mass_g:.2f}")

print(f"Number of Neutral current events: {n_nc_events*alt_3_sf:.2f}")
print(f"Number of CC electron neutrino events: {n_cc_el_events*alt_3_sf:.2f}")
print(f"Number of CC muon neutrino events: {n_cc_mu_events*alt_3_sf:.2f}")
print(f"Number of CC tau neutrino events: {n_cc_tau_events*alt_3_sf:.2f}")
print("\n")

print(f"Nominal target mass = {tungsten_mass_g/1000:.2f} kg")

Alternative 2 - 45 SCT | 1000 kg Tungsten | 150 mm
------------------------------------------------------------
New mass / nominal = 5.80
Number of Neutral current events: 6265.47
Number of CC electron neutrino events: 3123.75
Number of CC muon neutrino events: 17223.51
Number of CC tau neutrino events: 70.70


Nominal target mass = 996.60 kg
